<a href="https://colab.research.google.com/github/ByungjunKim/ModernKoreanSubword/blob/main/ModernKoreanSubword.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 근대 한국어 서브워드 형태소 분석기

In [1]:
# kiwi 설치
!pip install -U -q kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 29.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 18.8 MB/s eta 0:00:00


In [2]:
#  서브워드 토크나이저 json 파일 다운로드
!git clone https://github.com/ByungjunKim/ModernKoreanSubword

Cloning into 'ModernKoreanSubword'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 90 (delta 47), reused 38 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (90/90), 5.88 MiB | 7.68 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [3]:
# kiwi 로드
from kiwipiepy import Kiwi
kiwi = Kiwi()
from kiwipiepy.sw_tokenizer import SwTokenizer
import os
from collections import Counter
from itertools import islice

### 1. 서브워드 토크나이저 확인

In [4]:
# 토크나이저 설정
tokenizer = SwTokenizer('./ModernKoreanSubword/tokenizer/241112_vo32000_tokenizer.json', kiwi=kiwi)

In [5]:
# 국민신보시대(國民新報時代)에정합방설(政合邦說)을제창(提唱)하야일한연방(日韓聯邦)하기로주론(主論)하기도경(經)하얏고
tokenizer.encode('國民新報時代에政合邦說을提唱하야日韓聯邦하기로主論하기도經하얏고')

array([2189,  173,  618, 1057,    3,  759,  425, 3759, 1159,    5, 9948,
          1,   17,  169, 1044, 4755, 3759,    1,   52,   24,  507,  549,
          1,   52,   27, 1499, 2805], dtype=uint32)

In [6]:
# 토크나이저 내장 단어 목록 확인
list(tokenizer.vocab.items())[120:140]

[('##기', 120),
 ('##上', 121),
 ('##명', 122),
 ('##後', 123),
 ('##은', 124),
 ('것', 125),
 ('##及', 126),
 ('다고/E', 127),
 ('·', 128),
 ('##나', 129),
 ('##로', 130),
 ('##問題', 131),
 ('##山', 132),
 ('##인', 133),
 ('가지/V', 134),
 ('##八', 135),
 ('라는/E', 136),
 ('##에', 137),
 ('##七', 138),
 ('##의', 139)]

In [8]:
# 토크나이징 함수 정의
def tokenize(sentence):
  vocab_list = tokenizer.encode(sentence)
  return [tokenizer.id2vocab[i].lstrip('##') for i in vocab_list] # 서브워드 표기('##') 제거

In [9]:
tokenize('國民新報時代에政合邦說을提唱하야日韓聯邦하기로主論하기도經하얏고')

['國民',
 '新',
 '報',
 '時代',
 '에/J',
 '政',
 '合',
 '邦',
 '說',
 '을/J',
 '提唱',
 '하/V',
 '야/E',
 '日',
 '韓',
 '聯',
 '邦',
 '하/V',
 '기/E',
 '로/J',
 '主',
 '論',
 '하/V',
 '기/E',
 '도/J',
 '經',
 '하얏고']

### 2. 여러 토크나이저 성능 비교

In [10]:
# 토크나이저 파일명 목록
tokenizer_files = ['vo32000', 'vo48000', 'vo64000'] # 최대 단어수
base_path = './ModernKoreanSubword/tokenizer/'

In [11]:
# 같은 문장에 대해 각 토크나이저로 토크나이징 결과 확인
target_sentence = '國民新報時代에政合邦說을提唱하야日韓聯邦하기로主論하기도經하얏고'

tokenizer_results = {}
for tokenizer_file in tokenizer_files:
    tokenizer_path = os.path.join(base_path, f"241112_{tokenizer_file}_tokenizer.json")
    tokenizer = SwTokenizer(tokenizer_path, kiwi=kiwi)
    vocab_list = tokenizer.encode(target_sentence)
    tokens = [tokenizer.id2vocab[i].lstrip('##') for i in vocab_list]
    tokenizer_results[tokenizer_file] = tokens

In [12]:
# 토크나이징 결과 출력 및 비교
for tokenizer_file, tokens in tokenizer_results.items():
    print(f"토크나이저 {tokenizer_file} 결과: {tokens}\n")

토크나이저 vo32000 결과: ['國民', '新', '報', '時代', '에/J', '政', '合', '邦', '說', '을/J', '提唱', '하/V', '야/E', '日', '韓', '聯', '邦', '하/V', '기/E', '로/J', '主', '論', '하/V', '기/E', '도/J', '經', '하얏고']

토크나이저 vo48000 결과: ['國民', '新', '報', '時代', '에/J', '政', '合', '邦', '說', '을/J', '提唱', '하/V', '야/E', '日', '韓', '聯邦', '하/V', '기/E', '로/J', '主', '論', '하/V', '기/E', '도/J', '經', '하얏고']

토크나이저 vo64000 결과: ['國民', '新報', '時代', '에/J', '政', '合', '邦', '說', '을/J', '提唱', '하/V', '야/E', '日', '韓', '聯邦', '하/V', '기/E', '로/J', '主', '論', '하/V', '기/E', '도/J', '經', '하얏고']



In [13]:
# 토크나이저 간의 결과 비교
tokenizer_pairs = [(tokenizer_files[i], tokenizer_files[i + 1]) for i in range(len(tokenizer_files) - 1)]
for file1, file2 in tokenizer_pairs:
    diff = set(tokenizer_results[file1]) ^ set(tokenizer_results[file2])
    if diff:
        print(f"토크나이저 {file1}과 {file2}의 다른 토큰들: {list(diff)}\n")

토크나이저 vo32000과 vo48000의 다른 토큰들: ['聯', '聯邦']

토크나이저 vo48000과 vo64000의 다른 토큰들: ['新', '新報', '報']



In [14]:
# 토크나이저 간의 결과 비교
tokenizer_pairs = [(tokenizer_files[i], tokenizer_files[i + 1]) for i in range(len(tokenizer_files) - 1)]
for file1, file2 in tokenizer_pairs:
    diff_tokens = set(tokenizer_results[file1]) ^ set(tokenizer_results[file2])
    file1_only = [token for token in tokenizer_results[file1] if token not in tokenizer_results[file2]]
    file2_only = [token for token in tokenizer_results[file2] if token not in tokenizer_results[file1]]
    if diff_tokens:
        print(f"토크나이저 {file1}과 {file2}의 비교 결과:")
        print(f"  토크나이저 {file1}에만 포함된 토큰들: {file1_only}")
        print(f"  토크나이저 {file2}에만 포함된 토큰들: {file2_only}\n")

토크나이저 vo32000과 vo48000의 비교 결과:
  토크나이저 vo32000에만 포함된 토큰들: ['聯']
  토크나이저 vo48000에만 포함된 토큰들: ['聯邦']

토크나이저 vo48000과 vo64000의 비교 결과:
  토크나이저 vo48000에만 포함된 토큰들: ['新', '報']
  토크나이저 vo64000에만 포함된 토큰들: ['新報']



In [16]:
# 각 토크나이저에 고유하게 등록된 단어 비교
unique_words = {}
all_words = Counter()

# 모든 단어 수집 및 고유 단어 확인
tokenizer_vocabularies = {}
for tokenizer_file in tokenizer_files:
    tokenizer_path = os.path.join(base_path, f"241112_{tokenizer_file}_tokenizer.json")
    tokenizer = SwTokenizer(tokenizer_path, kiwi=kiwi)
    vocab = tokenizer.vocab  # 전체 단어 목록 가져오기 (dict 형태)
    processed_vocab = {word.lstrip('##') for word in vocab.keys()}  # ## 제거
    tokenizer_vocabularies[tokenizer_file] = processed_vocab
    all_words.update(processed_vocab)

for tokenizer_file, vocab in tokenizer_vocabularies.items():
    unique_words[tokenizer_file] = [word for word in vocab if all_words[word] == 1]

In [17]:
# 고유 단어 결과 출력
for tokenizer_file, words in unique_words.items():
    print(f"토크나이저 {tokenizer_file}에 고유하게 등록된 단어 수: {len(words)}")
    print(f"고유 단어 예시: {words[:30]}\n")

토크나이저 vo32000에 고유하게 등록된 단어 수: 23
고유 단어 예시: ['融緩漫', '꽁하/V', '소박하/V', '事次本社來', '大히擧行하얏', '해도재녕', '滿拓殖', '산상공회의소', 'oth', '立尋常', '在滿機構改', 'pane', '濟뿔럭', '次五個年計', '活必需', '南利原郡', '一志二片三', '洋拓殖', '소를뎨긔', '勞農政', '멋대', 'ationa', '鮮博覽會']

토크나이저 vo48000에 고유하게 등록된 단어 수: 22
고유 단어 예시: ['동학교', '錢四等二十圓', '國委員會', '錢四等二十一圓', '第七號違', '俉', '板垣陸', '珝', '語奉答文', '專外科李容', '桯', '盯', '重奏', '奉答', '럿케', '호별', '笳', '条', '狑', '祝', '稻田', '十九分']

토크나이저 vo64000에 고유하게 등록된 단어 수: 10989
고유 단어 예시: ['칵', '趣意', '談判', '三萬餘圓', '팽창', '空相', '을演', '富山', '至當', '織造', '엥', '六萬五千', '全滿', '戰地', '스파이', '을떠나', '一般이', '四百六十', '會席上', '한섬', '鮮銀總裁', '져셔', '몬로', '자아', '정류장', '발작', '그方法', '할수잇슬', '투숙', '十一圓臺']

